<a href="https://colab.research.google.com/github/yejianfeng2014/AI/blob/master/%E6%A8%A1%E5%9E%8B%E5%9F%BA%E6%9C%AC%E8%AE%AD%E7%BB%83_%E4%BF%9D%E5%AD%98_%E4%B8%8B%E8%BD%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

挂载云硬盘，如果验证数据请填写验证信息 挂载硬盘，**只需要挂载一次**

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#训练一个基本模型，采用mnist 数据集，训练了一个模型
## 关键点，实现了模型的保存到云网盘上

1.     model_path = "./test/model.ckpt" 核 必须以‘./’ 开头，就会保存在云盘的根目录上。然后第二个test会创建一个目录把模型直接保存进去列表项
2.  然后选择文件 ->保存 会自动保存在云网盘下test的目下，就可以在云网盘里面下载
    

In [2]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

# Parameters
learning_rate = 0.001
batch_size = 100
display_step = 1
model_path = "./test/model.ckpt"

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [4]:
saver = tf.train.Saver()
print("Starting 1st session...")


Starting 1st session...


In [5]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(20):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("First Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    # Save model weights to disk
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)

# Running a new session
print("Starting 2nd session...")

Epoch: 0001 cost= 148.555580732
Epoch: 0002 cost= 37.329880381
Epoch: 0003 cost= 23.441558777
Epoch: 0004 cost= 16.364971469
Epoch: 0005 cost= 12.042239525
Epoch: 0006 cost= 8.769514230
Epoch: 0007 cost= 6.625274057
Epoch: 0008 cost= 4.867614367
Epoch: 0009 cost= 3.569638941
Epoch: 0010 cost= 2.792400248
Epoch: 0011 cost= 2.008744070
Epoch: 0012 cost= 1.505613955
Epoch: 0013 cost= 1.182687648
Epoch: 0014 cost= 0.903851427
Epoch: 0015 cost= 0.697083753
Epoch: 0016 cost= 0.642139443
Epoch: 0017 cost= 0.567303761
Epoch: 0018 cost= 0.435636582
Epoch: 0019 cost= 0.440558042
Epoch: 0020 cost= 0.324702794
First Optimization Finished!
Accuracy: 0.9528
Model saved in file: ./test/model.ckpt
Starting 2nd session...


使用以下方法一次可以下载一个文件下来，其他几个也可以采用类似的方法下载。
我还是建议在云网盘里面使用客户端下载，毕竟网页下载很慢。

In [0]:
from google.colab import files
files.download('./test/model.ckpt.index') 

